In [69]:
import pandas as pd
import matplotlib.pyplot as plt

In [70]:
# the default first 5 indexes of headers
header_first_5 = ['nonsense', '年份', '电网区域', '省份', '省下区域', 'Tech']

In [71]:
class DataFromSheet:
    def __init__(self, path_to_sheet: str):
        self.path = path_to_sheet
        self.dataframe = pd.read_excel(self.path)

In [72]:
class WorkingData(DataFromSheet):
    def __init__(self, path_to_sheet: str, columns: list, focused_index: str, tech_fuel_dict: dict):
        # read the Excel file
        super().__init__(path_to_sheet)
        # passing the value of the member variables
        self.working_df = None
        self.columns = header_first_5 + columns
        self.focused_index = focused_index
        self.tech_fuel_dict = tech_fuel_dict

        # pre-clean the data
        self.dataframe = self.read_data()

    def read_data(self):
        self.dataframe.columns = self.columns   # set the column names of the dataframe
        self.dataframe = self.dataframe[header_first_5+[self.focused_index]]  # select the column of the interested index
        self.dataframe['Fuel_Group'] = self.dataframe['Tech'].map(self.tech_fuel_dict)   # map the fuel group value based on the passed dict
        # self.dataframe = self.dataframe[(self.dataframe[self.focused_index] != 0.0)].dropna()    # drop empty columns

    def rule_out(self, regional_agg_level: str, selected_regions: list, select_reverse=True):
        dataframe = self.dataframe.copy()

        # aggregate data to the given regional agg level
        if not regional_agg_level == '全国':
            dataframe = dataframe.groupby(by=['年份', 'Fuel_Group', regional_agg_level]).sum().reset_index()
            if not select_reverse:
                dataframe = dataframe[dataframe[regional_agg_level].isin(selected_regions)]
            else:
                dataframe = dataframe[~dataframe[regional_agg_level].isin(selected_regions)]

        else:
            # if aggregate to national
            dataframe = dataframe.groupby(by=['年份', 'Fuel_Group']).sum().reset_index()

        self.working_df = dataframe
        return dataframe

In [73]:
class Utils(DataFromSheet):
    def __init__(self, path_to_sheet):
        super().__init__(path_to_sheet)
        self.color_scheme = self.read_color_scheme()
        self.tech_fuel_group_dict = self.read_fuel_group()

    def read_color_scheme(self)->dict:
        """
        get the color assigned for each bar
        :return: Dict
        """
        color_scheme = dict(self.dataframe[['Fuel_Group', "HEX"]].values)
        return color_scheme

    def read_stack_order(self, tech: list)->list:
        """
        :param tech: list of names of technologies (unique) appeared in sheet
        :return: the ordered list of header for stacked plot
        """
        # sort the fuel group by the specified order
        order_list = self.dataframe.dropna().sort_values(by='Order', ascending=True)['Fuel_Group'].unique().tolist()

        ls = []
        for group in order_list:
            if group in tech:
                ls = ls.append(group)

        return ls

    def read_fuel_group(self)->dict:
        """
        read the dictionary of tech and fuel groups
        :return: the dictionary used for conversion
        """
        return dict(self.dataframe[['Tech', 'Fuel_Group']].values)

In [ ]:
class DiagramCapability:


In [74]:
path_to_data = '/Users/zhixuan/PycharmProjects/3e-data-wiz/example-files/情景1产能.xlsx'
path_to_ref = '/Users/zhixuan/PycharmProjects/3e-data-wiz/example-files/color_index.xlsx'
columns = ["Installed Power Capacity (MW)", "Installed Heat Capacity (MW)", "Installed Hydrogen Production Capacity (MW)"]
focused_index = 'Installed Power Capacity (MW)'
regional_agg_level = '省份'
utils = Utils(path_to_ref)
WorkingData(path_to_data, columns).rule_out(focused_index, utils.tech_fuel_group_dict, regional_agg_level)

,年份,Fuel_Group,省份,Installed Power Capacity (MW)
0,2020.0,SMR,Hebei,0.000000
1,2020.0,光热,Beijing,1.000000
2,2020.0,光热,Gansu,210.000000
3,2020.0,光热,Hainan,10.000000
4,2020.0,光热,Qinghai,210.000000
...,...,...,...,...
986,2035.0,集中式光伏,West-InnerMongolia,7634.000000
987,2035.0,集中式光伏,Xinjiang,12340.000000
988,2035.0,集中式光伏,Xizang,1370.000000
989,2035.0,集中式光伏,Yunnan,47320.324480
